## [week6-lab & Homework #1] airpollution1.txt, airpollution2.txt, mart1.txt, mart2.txt 파일에 대해서 아래 질문에 답하시오. 

1. airpollution1.txt 파일을 읽어서 "wonju" 를 제외한 나머지 도시의 날짜와 PM2포함하는 RDD를 생성하고 내용을 출력하는 프로그램을 작성하라.
2. airpollution1.txt, airpollution2.txt에 대해서 각 도시의 년월 별 PM25의 평균값을 구하고 출력하는 프로그램을 작성하라.
2. airpollution1.txt, airpollution2.txt에 대해서 각 도시별 전체 기간 평균 PM25, PM10 평균값을 구하고 출력하는 프로그램을 작성하라.
3. mart.txt를 입력으로 해서 <실행결과>와 같은 내용의 RDD를 생성하는 프로그램을 작성하라. 

In [1]:
from pyspark.sql import SparkSession

In [25]:
spark_session = SparkSession.builder.master("local").appName("week6").getOrCreate()
airpollution1 = spark_session.sparkContext.textFile("airpollution1.txt")
filtered_data = airpollution1.map(lambda line: line.split(',')) \
                             .filter(lambda x: x[3] != 'wonju') \
                             .map(lambda x: (x[0], float(x[1]), x[3]))

print(filtered_data.collect())

[('2017.05', 5.2, 'chungju'), ('2017.06', 5.19, 'chungju'), ('2017.05', 5.2, 'jecheon'), ('2017.06', 5.19, 'jecheon')]


In [26]:
def average_pm25(rdd):
    return rdd.map(lambda line: line.split(',')) \
              .map(lambda x: ((x[3], x[0]), (float(x[1]), 1))) \
              .reduceByKey(lambda a, b: (a[0] + b[0], a(11) + b(11))) \
              .mapValues(lambda v: v[0]/v[1])
airpollution2 = spark_session.sparkContext.textFile("airpollution2.txt")
avg_pm25_1 = average_pm25(airpollution1)
avg_pm25_2 = average_pm25(airpollution2)
print(avg_pm25_1.collect())
print(avg_pm25_2.collect())

[(('chungju', '2017.05'), 5.2), (('chungju', '2017.06'), 5.19), (('jecheon', '2017.05'), 5.2), (('jecheon', '2017.06'), 5.19), (('wonju', '2017.05'), 5.2), (('wonju', '2017.06'), 5.19)]
[(('chungju', '2020.05'), 6.2), (('chungju', '2020.06'), 6.21), (('jecheon', '2020.05'), 7.33), (('jecheon', '2020.06'), 8.67), (('wonju', '2020.05'), 3.31), (('wonju', '2020.06'), 8.22)]


In [27]:
def parse_line(line):
    parts = line.split(',')
    return (parts[3], (float(parts[1]), float(parts[2]), 1))
def overall_average(rdd):
    return rdd.map(parse_line) \
              .reduceByKey(lambda a, b: ((a[0] + b[0]), 
                                         (a[1] + b[1]), 
                                         a[2] + b[2])) \
              .mapValues(lambda v: {'avg_pm25': v[0]/v[2], 'avg_pm10': v[1]/v[2]})
combined_data = airpollution1.union(airpollution2)
overall_avg = overall_average(combined_data)
print("Average PM25 and PM10 for each city:")
print(overall_avg.collect())

Average PM25 and PM10 for each city:
[('chungju', {'avg_pm25': 5.7, 'avg_pm10': 4.0}), ('wonju', {'avg_pm25': 5.48, 'avg_pm10': 4.5}), ('jecheon', {'avg_pm25': 6.5975, 'avg_pm10': 4.5})]


In [28]:
def parse_line(line):
    parts = line.split(',')
    return (parts[0], (','.join(parts[1:]), len(parts[1:])))
mart1 = spark_session.sparkContext.textFile("mart1.txt")
mart2 = spark_session.sparkContext.textFile("mart2.txt")
combined_data = mart1.union(mart2)
result = combined_data.map(parse_line).reduceByKey(lambda a, b: (a[0] + ',' + b[0], a[1] + b[1]))
result_list = result.sortBy(lambda x: x[0]).collect()
print(result_list)

[('1', ('오렌지,사이다,세척제,사이다,맥주,사이다', 6)), ('2', ('우유,오렌지,세척제,기저귀,맥주,기저귀,맥주', 7)), ('3', ('오렌지,세재,우유,빵,세재', 5)), ('4', ('오렌지,세제,사이다', 3)), ('5', ('우유,빵,세척제,사이다', 4)), ('7', ('오렌지,빵,오렌지', 3))]


In [29]:
def parse_line(line):
    parts = line.split(',')
    return (parts[0], (','.join(parts[1:]), len(parts[1:])))
mart1 = spark_session.sparkContext.textFile("mart1.txt")
mart2 = spark_session.sparkContext.textFile("mart2.txt")
combined_data = mart1.union(mart2)
result = combined_data.map(parse_line).reduceByKey(lambda a, b: (a[0] + ',' + b[0], a[1] + b[1]))
desired_order = ['4', '1', '5', '3', '2', '7']
sorted_result_list = [next(item for item in result.collect() if item[0] == id) for id in desired_order]
print(sorted_result_list)

[('4', ('오렌지,세제,사이다', 3)), ('1', ('오렌지,사이다,세척제,사이다,맥주,사이다', 6)), ('5', ('우유,빵,세척제,사이다', 4)), ('3', ('오렌지,세재,우유,빵,세재', 5)), ('2', ('우유,오렌지,세척제,기저귀,맥주,기저귀,맥주', 7)), ('7', ('오렌지,빵,오렌지', 3))]
